# Stock Price Prediction with Deep Learning
#
# This notebook implements a stock price prediction model using an LSTM neural network. The model incorporates technical indicators like EMAs, MACD, Bollinger Bands, and support/resistance levels.
#
# Overview
# 1. **Imports and Setup**: Import necessary libraries.
# 2. **Function Definitions**: Define functions for data retrieval, preprocessing, model creation, and prediction.
# 3. **User Inputs**: Collect user inputs for stock symbol, date range, and other parameters.
# 3a. **Data Retrieval and Preprocessing**: Fetch and prepare the stock data.
# 3b. **Model Training**: Train the LSTM model.
# 3c. **Predictions and Visualization**: Generate predictions and plot the results.

# 1. Imports and Setup
# Import all required libraries for data handling, including `yfinance` for data, `tensorflow` for machine learning and `ipywidgets` for interactivity.

In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
# Ensure plots display inline in Jupyter Notebook
%matplotlib inline  



# 2. Function Definitions
# Define functions for fetching stock data, preparing data, building the model, predicting future prices, and plotting results.

In [2]:
def get_stock_data(ticker, start_date, end_date, sr_window):
    """
    Fetch stock data and calculate technical indicators.
    
    Args:
        ticker (str): Stock symbol (e.g., 'AAPL')
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        sr_window (int): Window size for support/resistance calculation
    
    Returns:
        df (DataFrame): Processed stock data
        yearly_levels (dict): Yearly support and resistance levels
    """
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    df = pd.DataFrame()
    df['Close'] = stock_data['Close']
    df['Volume'] = stock_data['Volume']
    df['EMA10'] = stock_data['Close'].ewm(span=10, adjust=False).mean()
    df['EMA20'] = stock_data['Close'].ewm(span=20, adjust=False).mean()
    df['EMA50'] = stock_data['Close'].ewm(span=50, adjust=False).mean()
    df['EMA200'] = stock_data['Close'].ewm(span=200, adjust=False).mean()
    
    # MACD
    ema12 = stock_data['Close'].ewm(span=12, adjust=False).mean()
    ema26 = stock_data['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # Bollinger Bands
    df['BB_Middle'] = stock_data['Close'].rolling(window=20).mean()
    df['BB_Std'] = stock_data['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
    df['BB_Lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)
    
    # Rolling window Support/Resistance
    df['Support'] = stock_data['Close'].rolling(window=sr_window, center=True).min()
    df['Resistance'] = stock_data['Close'].rolling(window=sr_window, center=True).max()
    
    # Calculate yearly support and resistance levels
    df['Year'] = df.index.year
    yearly_levels = {}
    for year in df['Year'].unique():
        year_data = df[df['Year'] == year]['Close']
        if not year_data.empty:
            yearly_levels[year] = {
                'Support': year_data.min(),
                'Resistance': year_data.max()
            }
    
    return df.dropna(), yearly_levels

def prepare_data(data, look_back=60):
    """
    Prepare data for LSTM model by scaling and creating sequences.
    
    Args:
        data (DataFrame): Stock data with features
        look_back (int): Number of previous days to use for prediction
    
    Returns:
        X_train, X_test, y_train, y_test, scaler, scaled_data, features
    """
    features = ['Close', 'Volume', 'EMA10', 'EMA20', 'EMA50', 'EMA200', 'MACD', 'MACD_Signal', 
                'BB_Upper', 'BB_Lower', 'Support', 'Resistance']
    scaled_data = MinMaxScaler(feature_range=(0, 1)).fit_transform(data[features])
    
    X, y = [], []
    for i in range(look_back, len(scaled_data)):
        X.append(scaled_data[i-look_back:i])
        y.append(scaled_data[i, 0])  # Predict Close price
    
    X, y = np.array(X), np.array(y)
    
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[0:train_size], X[train_size:]
    y_train, y_test = y[0:train_size], y[train_size:]
    
    return X_train, X_test, y_train, y_test, MinMaxScaler(feature_range=(0, 1)).fit(data[features]), scaled_data, features

def create_model(look_back=60, n_features=12):
    """
    Create and compile the LSTM model.
    
    Args:
        look_back (int): Number of previous days used as input
        n_features (int): Number of features in the input data
    
    Returns:
        model: Compiled Keras model
    """
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(look_back, n_features)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def predict_future(model, last_sequence, scaler, days_to_predict, look_back, n_features):
    """
    Predict future stock prices iteratively.
    
    Args:
        model: Trained Keras model
        last_sequence: Last sequence of data for prediction
        scaler: MinMaxScaler object
        days_to_predict (int): Number of days to predict
        look_back (int): Number of previous days used as input
        n_features (int): Number of features in the input data
    
    Returns:
        future_predictions: Array of predicted prices
    """
    future_predictions = []
    current_sequence = last_sequence.reshape((1, look_back, n_features))
    
    for _ in range(days_to_predict):
        next_pred = model.predict(current_sequence, verbose=0)
        future_predictions.append(next_pred[0, 0])
        
        new_row = np.zeros(n_features)
        new_row[0] = next_pred[0, 0]
        new_row[1:] = current_sequence[0, -1, 1:]
        current_sequence = np.roll(current_sequence, -1, axis=1)
        current_sequence[0, -1] = new_row
    
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_scaled = np.hstack([future_predictions, np.tile(current_sequence[0, -1, 1:], (days_to_predict, 1))])
    future_predictions = scaler.inverse_transform(future_scaled)[:, 0]
    return future_predictions

# 3. User Inputs with ipywidgets
# Create an interactive form using `ipywidgets` to collect user inputs for the stock symbol, date range, and other parameters.

In [ ]:
# Define widgets for user inputs
ticker_input = widgets.Text(
    value='',
    placeholder='Enter stock symbol (e.g., AAPL)',
    description='Stock Symbol:',
    layout={'width': '300px', 'description_width': '150px'}  
)

start_date_input = widgets.Text(
    value='',
    placeholder='YYYY-MM-DD',
    description='Start Date:',
    layout={'width': '300px', 'description_width': '150px'}  
)

end_date_input = widgets.Text(
    value='',
    placeholder='YYYY-MM-DD',
    description='End Date:',
    layout={'width': '300px', 'description_width': '150px'}  
)

sr_window_input = widgets.IntSlider(
    value=200,
    min=10,
    max=200,
    step=10,
    description='S/R Window:',
    layout={'width': '400px', 'description_width': '150px'}  
)

days_to_predict_input = widgets.IntSlider(
    value=30,
    min=1,
    max=90,
    step=1,
    description='Days to Predict:',
    layout={'width': '400px', 'description_width': '150px'}  
)

epochs_input = widgets.IntSlider(
    value=50,
    min=10,
    max=100,
    step=10,
    description='Training Epochs:',
    layout={'width': '400px', 'description_width': '150px'}  
)

run_button = widgets.Button(
    description='Run Analysis',
    button_style='success',
    tooltip='Click to run the analysis',
    icon='play'
)

# Output widget to display results
output = widgets.Output()

# Define the function to run when the button is clicked
def on_run_button_clicked(b):
    with output:
        clear_output()  # Clear previous output
        
        # Get user inputs
        ticker = ticker_input.value.upper()
        start_date = start_date_input.value
        end_date = end_date_input.value
        sr_window = sr_window_input.value
        days_to_predict = days_to_predict_input.value
        epochs = epochs_input.value
        
        # Fixed parameters
        look_back = 60
        n_features = 12
        
        # Fetch and prepare data
        print("Fetching and preparing data...")
        stock_data, yearly_levels = get_stock_data(ticker, start_date, end_date, sr_window)
        X_train, X_test, y_train, y_test, scaler, scaled_data, features = prepare_data(stock_data, look_back)
        
        # Display the first few rows of the data
        print("First few rows of the stock data:")
        display(stock_data.head())
        
        # Create and train model
        print("Training the model...")
        model = create_model(look_back, n_features)
        model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_split=0.1)
        
        # Display model summary
        print("Model Summary:")
        model.summary()
        
        # Make predictions on historical data
        print("Generating predictions...")
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)
        
        # Inverse transform historical predictions
        train_predict = scaler.inverse_transform(np.hstack([train_predict, np.zeros((train_predict.shape[0], n_features-1))]))[:, 0]
        y_train_inv = scaler.inverse_transform(np.hstack([y_train.reshape(-1, 1), np.zeros((y_train.shape[0], n_features-1))]))[:, 0]
        test_predict = scaler.inverse_transform(np.hstack([test_predict, np.zeros((test_predict.shape[0], n_features-1))]))[:, 0]
        y_test_inv = scaler.inverse_transform(np.hstack([y_test.reshape(-1, 1), np.zeros((y_test.shape[0], n_features-1))]))[:, 0]
        
        # Predict future prices
        last_sequence = scaled_data[-look_back:]
        future_predictions = predict_future(model, last_sequence, scaler, days_to_predict, look_back, n_features)
        
        # Generate future dates
        last_date = stock_data.index[-1]
        future_dates = pd.date_range(start=last_date, periods=days_to_predict + 1, freq='B')[1:]
        
        # Get recent rolling window support and resistance levels
        support_level = stock_data['Support'].iloc[-1]
        resistance_level = stock_data['Resistance'].iloc[-1]
        
        # Plotting
        print("Plotting results...")
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), gridspec_kw={'height_ratios': [3, 1]}, sharex=True)
        
        # Price plot
        ax1.plot(stock_data.index[look_back:len(train_predict)+look_back], train_predict, 
                 label='Training Predictions')
        ax1.plot(stock_data.index[len(train_predict)+look_back:], test_predict, 
                 label='Test Predictions')
        ax1.plot(stock_data.index, stock_data['Close'], label='Actual Prices', color='darkgreen')
        ax1.plot(future_dates, future_predictions, label='Future Predictions', color='blue', linestyle='--')
        ax1.plot(stock_data.index, stock_data['EMA200'], label='EMA200', color='steelblue', linestyle='-.', alpha=0.5)
        ax1.plot(stock_data.index, stock_data['EMA50'], label='EMA50', color='coral', linestyle='-.', alpha=0.5)
        
        # Plot rolling window support and resistance
        ax1.axhline(y=support_level, color='lime', linestyle='-', alpha=0.7, label=f'Support ({sr_window}-day)')
        ax1.axhline(y=resistance_level, color='crimson', linestyle='-', alpha=0.7, label=f'Resistance ({sr_window}-day)')
        
        # Plot yearly support and resistance levels
        colors = ['pink', 'white', 'gold', 'teal']
        color_idx = 0
        for year, levels in yearly_levels.items():
            support = levels['Support']
            resistance = levels['Resistance']
            color = colors[color_idx % len(colors)]
            ax1.axhline(y=support, color=color, linestyle='-', alpha=0.5, label=f'Support {year}')
            ax1.axhline(y=resistance, color=color, linestyle=':', alpha=0.5, label=f'Resistance {year}')
            color_idx += 1
        
        ax1.set_title(f'{ticker} Stock Price Prediction')
        ax1.set_ylabel('Price')
        ax1.legend()
        
        # MACD plot
        ax2.plot(stock_data.index, stock_data['MACD'], label='MACD', color='blue')
        ax2.plot(stock_data.index, stock_data['MACD_Signal'], label='Signal Line', color='orange')
        ax2.axhline(0, color='gray', linestyle='--', alpha=0.5)
        ax2.set_title('MACD')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('MACD Value')
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
        
        # Calculate and print RMSE
        train_rmse = np.sqrt(np.mean((train_predict - y_train_inv) ** 2))
        test_rmse = np.sqrt(np.mean((test_predict - y_test_inv) ** 2))
        print(f'Training RMSE: {train_rmse:.2f}')
        print(f'Testing RMSE: {test_rmse:.2f}')
        print(f'Support Level ({sr_window}-day window): ${support_level:.2f}')
        print(f'Resistance Level ({sr_window}-day window): ${resistance_level:.2f}')
        print('Yearly Support and Resistance Levels:')
        for year, levels in yearly_levels.items():
            print(f'{year}: Support ${levels["Support"]:.2f}, Resistance ${levels["Resistance"]:.2f}')
        print(f'Future Predictions for next {days_to_predict} days:')
        for date, price in zip(future_dates, future_predictions):
            print(f'{date.strftime("%Y-%m-%d")}: ${price:.2f}')

# Connect the button to the function
run_button.on_click(on_run_button_clicked)

# Display the widgets
form = widgets.VBox([
    ticker_input,
    start_date_input,
    end_date_input,
    sr_window_input,
    days_to_predict_input,
    epochs_input,
    run_button,
    output
])
display(form)